## Example OMP_Stencial: Loop Optimization in OpenMP

This example is almost all by example in file [omp_stencil.c](./omp_stencil/stencil.c). This produces the timing results that show the performance benefits. Run this code without compiler optimization
```
gcc -fopenmp -O0 stencil.c 
clang -fopenmp -O0 stencil.c
```

This makes the results more understandable because it prevents the compiler from vectorizing the code and optimizing loops. If we run at a higher optimization level, the compiler will automatically make some of the optimizations that we want to demonstrate.

### Loop iteration order

**Goal**: Understanding how strided and sequential access influence performance and how this is related to cache-lines.

For 2-d dense data, the array must be serialized to memory, i.e. in a linear order.
The serialization strategies are named by which dimension (row versus column) 
occurs sequentially in memory.

<img src="https://upload.wikimedia.org/wikipedia/commons/4/4d/Row_and_column_major_order.svg" width=256 title="Row versus column major order." />

Choosing a memory efficient order for loops has a big impact on performance.
  * Successive loop iterations access adjacent elements or
  * Successive loop iterations access strided elements, i.e. one element every spaced out by the other elements of a row or column.
  
The different orders are also associated with programming languages that use these conventions.
  
<img src="https://images.slideplayer.com/23/6540072/slides/slide_3.jpg" width=512 title="from Edgar Gabriel at UH" />

C-Order is row major. Fortan order is column major. The colors indicate how the access patterns will be sequential or strided. Accessing succesive elements in a row (blue) is sequential in C-order and strided in Fortran-order.

There are many conventions about loop ordering and they get confusing.  Reason carefully about how the loops variables are enumerated and the data layout.  For example, images are almost always in Fortran order so that programming then in C looks weird.

### Sequential access in `stencil.c`

We provide two routines that show the difference between sequential and strided access in C.

Which of the following performs sequential access?

```c
void initializeyx ( double* array )
{
    /* Initialize the array to random values */
    for (int y=0; y<DIM; y++) {
        for (int x=0; x<DIM; x++) {
            array[x*DIM+y] = (double)rand()/RAND_MAX;
        }        
    }
}

void initializexy ( double* array )
{
    /* Initialize the array to random values */
    for (int x=0; x<DIM; x++) {
        for (int y=0; y<DIM; y++) {
            array[x*DIM+y] = (double)rand()/RAND_MAX;
        }        
    }
}
```

The performance difference reflects the latency difference between sequential and strided acceess. 

The way that I think about this is that:
  * when accessing data sequentially, every cache line fetch retrieves a 16 elements (for 128 byte cache lines)
  * when accessing data strided, every cache line fetch retrieves 1 element.

### A Parallel Stencil

A common pattern in numerical computing is to compute a [compact stencil](https://en.wikipedia.org/wiki/Compact_stencil). In this case, we evaluate a value at the red point as a function of its neighboring (black) points.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c5/CompactStencil.svg/300px-CompactStencil.svg.png" width=256 title="Compact stencil." />

The following function computes an average over a compact stencil at each (well defined) cell in a 2-d grid.  This computation pattern is used frequently in convolutional neural networks, graphics, spatial data processing, etc.

```c
void stencil_average ( double* input_ar, double* output_ar )
{
    double partial = 0.0;

    for (int x=HWIDTH; x<DIM-HWIDTH; x++) {
        for (int y=HWIDTH; y<DIM-HWIDTH; y++) {
            for (int xs=-1*HWIDTH; xs<=HWIDTH; xs++) {
                for (int ys=-1*HWIDTH; ys<=HWIDTH; ys++) {
                    partial += input_ar[DIM*(x+xs)+(y+ys)];
                }   
            }   
            output_ar[DIM*x+y] = partial/((2*HWIDTH+1)*(2*HWIDTH+1));
            partial=0.0;
        }       
    }
}
```

### Parallelizing `stencil_average`

**Goal**: Understand how to add `omp` directives to parallelize loops and why to parallelize the outer, rather than the inner, loop.

```c
void stencil_average_omp ( double* input_ar, double* output_ar )
{
    omp_set_num_threads(4);
    #pragma omp parallel for 
    for (int x=HWIDTH; x<DIM-HWIDTH; x++) {
        for (int y=HWIDTH; y<DIM-HWIDTH; y++) {
            double partial = 0.0;
```

To parallelize this computation, we need to do two things:
  (1) add parallel directives around the outer loop.
  (2) move the variable into the inner scope.

_Why the outer loop?_ This creates a groups of threads that divide the iterations of the outer most loop. The parallel context exists for the entire computation.  

_What happens if we parallelize the inner loop?_

```c
    // This is wrong
    for (int x=HWIDTH; x<DIM-HWIDTH; x++) {
        #pragma omp parallel for 
        for (int y=HWIDTH; y<DIM-HWIDTH; y++) {
            double partial = 0.0;
```

Threads are created and destroyed for each iteration of the outer loop. The overhead of thread creation will slow down the program and loose speedup.  Refer to the function `stencil_avg_omp_inner()`.  This effect can either be minor or major, depending upon how much work is done in the inner loop.

__Question__: How many threads are created when parallelizing the inner loop?  When parallelizing the outer loop?

### Loop Independence 

**Goal**: Understand the concept of loop independence and the performance consequences of sharing variables among threads. Connect the performance degradation to cache invalidation. 

For parallelism, we require that the iterations of the loops are _independent_, i.e. the computation of that iteration does not depend on prior iterations and does not affect subsequent iterations. This allows a loop to be run in parallel and produce the same result as the serial program.  We say that _loop independence_ is neccessary to achieve _serial equivalance_.

The code also moves the declaration and initialization of variable `partial` inside the loop. This is needed for correctness. The variable `partial` sums the contribution of the stencil for each iteration. By declaring partial inside the loop, each iteration has its own private variable. This also means that each thread has it's own copy of partial.  We say that this variable is _thread private_.

If we leave partial defined in the outer scope, it is a "shared variable" among threads (see `stencil_average_omp_bad`). The parallel threads update a single copy of the variable.  In addition to being incorrect, this has a negative performance affect, because the shared variable leads to _interference_ in the form _of memory contention_.

### False Sharing

**Goal**: Understand the concept of false sharing when threads share a cache line (even when they don't share variables).

The following image is credited to Intel and appears on https://d2l.ai/chapter_computational-performance/hardware.html

<img src="https://d2l.ai/_images/falsesharing.svg" width=512 />

Almost all figures that describe false sharing use this format, including the one I learned in graduate school. False sharing arises when two cores/processors update _different_ addresses in the same cache line. The hardware that manages cache coherency invalidates the cache line on the other processor.  This should take latency equal to the level in the hierarchy at which the sharing occurs. Using the i7 Nehalem numbers 
  * 35 clock cycles for two cores on same processor (L3)
  * 130 clock cycles for two cores on different processors (memory)
  
We demonstrate false sharing in `stencil_average_omp_false()`.  This version creates an array of `partial` on for each thread. The idea is that each thread will have its own copy that only it updates. It would seem that this would prevent the sharing problem seen in `stencil_average_omp_bad()`. However, it does not. The shared variables are contained within the same cache line and results in false sharing.

False sharing is real hazard in parallel programming. The programmer must ensure not only that each thread has it's own variable, but that those variables are not in the same cache line.  The best way to do this is to have local variables declared on each thread's private stack.

### Loop Unrolling

Loop unrolling is a time-space tradeoff typically made by compilers
  * time savings: eliminate branching instructions in evaluating loop conditional
  * space increase: make a bigger program with more statements

This example unrolls the entire stencil (5x5) eliminating the two inner loops.
    
```c
            partial = input_ar[DIM*(x-2)+(y-2)];
            partial += input_ar[DIM*(x-2)+(y-1)];
            partial += input_ar[DIM*(x-2)+(y)];
            partial += input_ar[DIM*(x-2)+(y+1)];
            partial += input_ar[DIM*(x-2)+(y+2)];

            partial += input_ar[DIM*(x-1)+(y-2)];
            partial += input_ar[DIM*(x-1)+(y-1)];
            partial += input_ar[DIM*(x-1)+(y)];
            partial += input_ar[DIM*(x-1)+(y+1)];
            partial += input_ar[DIM*(x-1)+(y+2)];

            partial += input_ar[DIM*(x)+(y-2)];
            partial += input_ar[DIM*(x)+(y-1)];
            partial += input_ar[DIM*(x)+(y)];
            partial += input_ar[DIM*(x)+(y+1)];
            partial += input_ar[DIM*(x)+(y+2)];

            partial += input_ar[DIM*(x+1)+(y-2)];
            partial += input_ar[DIM*(x+1)+(y-1)];
            partial += input_ar[DIM*(x+1)+(y)];
            partial += input_ar[DIM*(x+1)+(y+1)];
            partial += input_ar[DIM*(x+1)+(y+2)];

            partial += input_ar[DIM*(x+2)+(y-2)];
            partial += input_ar[DIM*(x+2)+(y-1)];
            partial += input_ar[DIM*(x+2)+(y)];
            partial += input_ar[DIM*(x+2)+(y+1)];
            partial += input_ar[DIM*(x+2)+(y+2)];

            output_ar[DIM*x+y] = partial/((2*HWIDTH+1)*(2*HWIDTH+1));
            partial = 0.0;
```

The example code shows:
  * unrolling improves serial performance (`stencil_average_unrolled()`)
  * unrolling improves parallel performance (`stencil_average_omp_unrolled()`)
  
In both cases the benefit comes from reducing the number of instructions.
  * _What instructions are eliminated?_
  * _Approximately what fraction of instructions are eliminated_?

We say that this loop is _fully unrolled_ in that we have written all of its iterations sequentially. Loops can be partially unrolled. It's common to refer to a loop being unrolled _X times_ which means that X iterations of the original loop happen in each iteration of the unrolled loop. 

A loop that has been unrolled 4 times looks like.

```c
// original loop
for (i=0; i<n; i++)
{
    do_stuff(i)
}

// unrolled four times
for (i=0; i<n; i=i+4)
{
    do_stuff(i)
    do_stuff(i+1)
    do_stuff(i+2)
    do_stuff(i+3)    
}
```

### Loop Fusion

Goal: Understand loop fusion and the benefits of eliminating branching instructions and thread creation.

Another effective optimization. The concept is to do the work of multiple loops in a single loop. For serial code, this has the benefit of
* evaluating loop conditional variables once for all fused loops
  
In OpenMP, this has the additional benefit:
* create and destroy threads once for the fused loops, rather than in each loop

The code example implements a function that sums two arrays in parallel:

```c
    #pragma omp parallel for 
    for (int x=0; x<DIM; x++) {
        for (int y=0; y<DIM; y++) {
            output_ar[x*DIM+y] = input_ar1[x*DIM+y] + input_ar2[x*DIM+y];
        }        
    }
```

Now consider that we want to compute a stencil average on two arrays and then add the result.  This can be done with three separate function calls each that has its own loop:
```c
    stencil_average_omp(rand_ar1, avg_ar1);
    stencil_average_omp(rand_ar2, avg_ar2);
    array_sum_omp(avg_ar1, avg_ar2, sum_ar);
```

The results for `seperate loops` shows the performance of doing each loop independently in parallel.  We can _fuse_ these loops and compute the average and sum in a single loop with one function call

```c
void fused_stencil_sum_omp ( double* input_ar1, double* input_ar2, double* output_ar )
{
    omp_set_num_threads(4);
    #pragma omp parallel for 
    for (int x=HWIDTH; x<DIM-HWIDTH; x++) {
        for (int y=HWIDTH; y<DIM-HWIDTH; y++) {
            double partial1 = 0.0;
            double partial2 = 0.0;
            for (int xs=-1*HWIDTH; xs<=HWIDTH; xs++) {
                for (int ys=-1*HWIDTH; ys<=HWIDTH; ys++) {
                    partial1 += input_ar1[DIM*(x+xs)+(y+ys)];
                    partial2 += input_ar2[DIM*(x+xs)+(y+ys)];
                }   
            }   
            output_ar[DIM*x+y] = partial1/((2*HWIDTH+1)*(2*HWIDTH+1)) + partial1/((2*HWIDTH+1)*(2*HWIDTH+1));
            partial1=0.0;
            partial2=0.0;
        }       
    }
}

fused_stencil_sum_omp(rand_ar1, rand_ar2, sum_ar);
```

In addition to reducing branching and thread creation, this example reduces the total number of writes by coalescing the three writes from the multiple loops into a single write.

### Loop Fission

This optimization divides the work of a single loop into multiple loops. I have not built an example, because I can't find one that is natural and effective. Fission can be used to make the data references in a loop smaller so that it fits into a smaller cache.  It is important to know that this optimization exists. It is useful in rare circumstances.

The _tiling_ optimizations for multiple dimensional arrays (**Lecture XX**) are a similar optimization to loop fusion.

### Seperable Dependencies and Reduction

**Goal**: Understand the OpenMP reduction clause and how it avoids false sharing.

It is a common pattern to compute an aggregate quantity (mean, sum, maximum) in a loop. This is known as a _reduction_ because you are reduce a larger amount of data into a single quantity. The natural implementation of this uses a shared variable. This is **inefficient**.

```c
void max_el_shared ( double* input_ar )
{
    double max_el = 0;
    omp_set_num_threads(4);
    
    #pragma omp parallel for
    for (int x=0; x<DIM; x++) {
        for (int y=0; y<DIM; y++) {
            max_el = max_el > input_ar[x*DIM+y] ? max_el : input_ar[x*DIM+y]; 
        }        
    }
}
```

All parallel threads are reading and writing a single variable so that memory location must be shared between all threads either through L3 (multicore, single processor) or memory (SMP or NUMA).

One might observe that the dependency can be _seperated_ through the following process:
  * give each thread a private variable `thread_max_el`
  * compute a thread local maximum in each thread
  * after all threads complete take the maximum of all the thread local maximums

This can be done manually and requires care. Naive implementations often result in false sharing.

OpenMP provides a directive for _reduction_ that takes care of all the details.

```c
   #pragma omp parallel for reduction ( max: max_el )
   for (int x=0; x<DIM; x++) {
        for (int y=0; y<DIM; y++) {
            max_el = max_el > input_ar[x*DIM+y] ? max_el : input_ar[x*DIM+y]; 
        }        
    }
```
`reduction` describe the pattern and the clause specifies a reduction operator and the variable name. There are:
* Arithmetic reductions: +,*,-,max,min .
* Logical operator reductions in C: & && | || ^

All of these are seperable dependencies and OpenMP will compute the partial result in each thread and accumulate the final result after threads complete.  Again, we can see the performance benefit.

### Turn Optimization On

Rebuilding and running the code with optimization reveals that compiler optimizations are likely separating dependencies and unrolling loops.
```
gcc -fopenmp -O3 stencil.c 
```

### Loop Scheduling

This is really an aside. I just want you to know that it exists.

The full looping directive includes the specification of a scheduling directive and a chunk size
```c
#pragma omp parallel for schedule(kind [,chunk size])
```
in which schedule can be one of:
* Static -- divide loop into equal sized chunks
* Dynamic -- build internal work queue and dispatch blocksize at a time
* Guided -- dynamic scheduling with decreasing block size for load balance
* Auto -- compiler chooses from above
* Runtime -- runtime configuration chooses from above
